In [1]:
import shared.config as config
import shared.image_util as image_util
import pymongo
import numpy as np
import pandas as pd
import json
import re
import ast
import os
from PIL import Image, ExifTags, ImageOps
import pathlib

import logging
logger = logging.getLogger()


fullfilename = '/input/images/DC014_Doelenstraat/L Fotos/Hout/DC14_P00_H0000_701_2.JPG'
dir = '/output/tijdelijk/'

In [2]:
filename, file_extension = os.path.splitext(fullfilename)
filename

'/input/images/DC014_Doelenstraat/L Fotos/Hout/DC14_P00_H0000_701_2'

In [3]:
basename = os.path.basename(fullfilename)
basename_noext, file_extension = os.path.splitext(basename)
basename_noext

'DC14_P00_H0000_701_2'

In [4]:
os.path.join(dir, basename_noext + '.jpg')

'/output/tijdelijk/DC14_P00_H0000_701_2.jpg'

In [5]:
config.AIRFLOW_INPUT_IMAGES

'/input/images'

In [6]:
import re
if fullfilename.startswith(config.AIRFLOW_INPUT_IMAGES):
    s3 = re.sub('^' + config.AIRFLOW_INPUT_IMAGES, '', fullfilename)
s3

'/DC014_Doelenstraat/L Fotos/Hout/DC14_P00_H0000_701_2.JPG'

In [7]:
config.AIRFLOW_OUTPUT_MEDIA

'/output/archeomedia'

In [8]:
out_filename = config.AIRFLOW_OUTPUT_MEDIA + re.sub('^' + config.AIRFLOW_INPUT_IMAGES, '', fullfilename)
out_filename

'/output/archeomedia/DC014_Doelenstraat/L Fotos/Hout/DC14_P00_H0000_701_2.JPG'

In [9]:
filename_noext, file_extension = os.path.splitext(fullfilename)

file_id = re.sub('^' + config.AIRFLOW_INPUT_IMAGES, '', filename_noext) + '.jpg'

out_filename = config.AIRFLOW_OUTPUT_MEDIA + file_id
file_id

'/DC014_Doelenstraat/L Fotos/Hout/DC14_P00_H0000_701_2.jpg'

In [10]:
def adjustAndSaveFile(fullfilename, fs, collection):
    try:
        filename, file_extension = os.path.splitext(fullfilename)
        dir, filename = os.path.split(fullfilename)
        if file_extension.lower() not in config.IMAGE_EXTENSIONS:
            return None

        # If image is sfeerfoto then ignore to not add pictures of people
        if 'Sfeerfoto' in fullfilename:
            return None

        # Get projectcd
        projectcd = fullfilename.replace(config.AIRFLOW_INPUT_IMAGES + '/','')
        projectcd = re.search('^([A-Z0-9]+).*', projectcd).group(1)

        # First read image and make 3 versions with different sizes. And put them in a list 
        if 'pdf' in file_extension.lower():
            images = pdf2image.convert_from_path(fullfilename)
            image = images[0]
        else:
            image = Image.open(fullfilename, 'r')
        mime_type = ''

        image_dict_sml, image_dict_med, image_dict_big = putImageInGrid(image, fullfilename, fs, dir, projectcd)    

        # Insert a record with metadata
        return collection.insert_one({
            'fileName': filename, 'fullFileName': fullfilename, 'imageUUID': str(image_dict_big), 'imageMiddleUUID': str(image_dict_med), 'imageThumbUUID': str(image_dict_sml),
            'fileType': file_extension.lower(), 'directory': dir, 'mime_type': 'image/jpeg', 'projectcd': projectcd 
            }).inserted_id  

    except Exception as err:
        msg = "Onbekende fout bij het bewaren van image, met tekst: " + str(err)
        logger.error(msg)    
        raise Exception(msg) from err



def putImageInGrid(image: Image, fullfilename, fs, dir, projectcd):
    print('Dir: '+ dir)
    try:
        basename = os.path.basename(fullfilename)
        basename_noext, file_extension = os.path.splitext(basename)

        # Grayscale images need special attention for not distoring
        lzw = False
        if 'lzw' in basename.lower() and 'tif' in file_extension.lower():
            image = ImageOps.grayscale(image)
            lzw = True

        lst_images = []
        image_dict_big = image.copy()
        image_dict_big = {
            #'image': adjustImage(image_dict_big, config.IMAGE_SIZE_BIGGEST if not lzw else config.IMAGE_SIZE_LZW) # grayscale images can have more pixels
            'image': image_util.adjustImage(image_dict_big, config.IMAGE_SIZE_BIGGEST) # grayscale images can have more pixels
        }
        lst_images.append(image_dict_big)
        image_dict_med = image.copy()
        image_dict_med = {
            'image': image_util.adjustImage(image_dict_med, config.IMAGE_SIZE_MIDDLE, postcard=True)
        }
        lst_images.append(image_dict_med)
        img_sml = image_dict_med['image'].copy()
        img_sml.thumbnail(config.IMAGE_SIZE_THUMB)
        image_dict_sml = {
            'image': img_sml
        }
        lst_images.append(image_dict_sml)

        #Set dir and Create dir if not exists before saving file
        if config.AIRFLOW_INPUT_IMAGES in dir:
            dir = re.sub('^' + config.AIRFLOW_INPUT_IMAGES, '', dir)
        dir = dir.lstrip('/\\')
        print('Dir: ' + dir)                         
        print(config.AIRFLOW_OUTPUT_MEDIA)
        p = os.path.join(config.AIRFLOW_OUTPUT_MEDIA, dir)
        print(p)
        
        pathlib.Path(os.path.join(config.AIRFLOW_OUTPUT_MEDIA, dir)).mkdir(parents=True, exist_ok=True) 
        print(dir)

        # Loop over all versions and store them in the filestore
        file_id_sml = os.path.join(dir, basename_noext + '-sml.jpg')
        image_dict_sml['image'].save(os.path.join(config.AIRFLOW_OUTPUT_MEDIA, file_id_sml))

        file_id_med = os.path.join(dir, basename_noext + '-med.jpg')
        image_dict_med['image'].save(os.path.join(config.AIRFLOW_OUTPUT_MEDIA, file_id_med))

        file_id = os.path.join(dir, basename_noext + '.jpg')
        image_dict_big['image'].save(os.path.join(config.AIRFLOW_OUTPUT_MEDIA, file_id))

        
        #for img in lst_images:
        #    width = str(img['image'].size[0])
        #    height = str(img['image'].size[1])
        
        #    b = io.BytesIO()
        #    img['image'].save(b, "JPEG")
        #    b.seek(0)

        #    img['uuid'] = fs.put(b, content_type='image/jpeg', height=int(height), width=int(width), filename=basename, dir=dir, fullfilename=fullfilename, projectcd=projectcd)

        #return image_dict_sml['uuid'], image_dict_med['uuid'], image_dict_big['uuid']
        return file_id_sml, file_id_med, file_id

    except Exception as err:
        msg = f"Onbekende fout bij het in grid plaatsen van image {fullfilename}, met tekst: {str(err)}" 
        logger.error(msg)    
        raise Exception(msg) from err

In [11]:
myclient = pymongo.MongoClient(config.MONGO_URI)
filesdb = myclient[config.DB_FILES]
stagingdb = myclient[config.DB_STAGING]
col = filesdb[config.COLL_FILENAMES]

#import pathlib
#pathlib.Path(os.path.dirname(out_filename)).mkdir(parents=True, exist_ok=True) 

image_util.adjustAndSaveFile(fullfilename, None, col)

ObjectId('634192e99deaeaa82cfe9b0f')

In [12]:
os.path.join(config.AIRFLOW_OUTPUT_MEDIA, '/dir/123')

'/dir/123'

In [13]:
os.sep

'/'

In [8]:
flds_migratie_info = ("Migratie-informatie", {"fields": ["soort","brondata","uuid"],"expanded": False})

show_fieldsets = [
    ("Projectvelden", {"columns": [
        {"fields": ["project", "vondst", "subnr", "aantal", "artefactsoort", "abr_materiaal", "sub_abr_materiaal", "typevoorwerp", "typecd", "functievoorwerp", "versiering", "beschrijving", "opmerkingen", "doos"], "grid":6},        
        {"fields": ["fotos"], "grid":6, "fulldisplay": True},        
    ]}),        
    ("Algemene Artefactvelden", {"columns": [
        {"fields": ["origine", "conservering", "exposabel", "restauratie", "literatuur", "publicatiecode", "catalogus", "weggegooid"], "grid":6},        
        {"fields": ["aantal", "gewicht", "afmetingen", "formaat_horizontaal", "formaat_vericaal", "compleetheid", "mai", "diversen"], "grid":6},        
    ]}),
    ("Datering", {"columns": [
        {"fields": ["datering_vanaf", "datering_tot", "artefactdatering_vanaf", "artefactdatering_tot"], "grid":6},        
        {"fields": ["vondstdatering_vanaf", "vondstdatering_tot", "spoordatering_vanaf", "spoordatering_tot"], "grid":6},        
    ]}),
flds_migratie_info]

show_fieldsets[0][1]['columns'][0]

{'fields': ['project',
  'vondst',
  'subnr',
  'aantal',
  'artefactsoort',
  'abr_materiaal',
  'sub_abr_materiaal',
  'typevoorwerp',
  'typecd',
  'functievoorwerp',
  'versiering',
  'beschrijving',
  'opmerkingen',
  'doos'],
 'grid': 6}

In [12]:
projs = show_fieldsets.pop(0)
show_fieldsets[0] = (projs[0], projs[1]['columns'][0])

show_fieldsets

[('Projectvelden',
  {'fields': ['project',
    'vondst',
    'subnr',
    'aantal',
    'artefactsoort',
    'abr_materiaal',
    'sub_abr_materiaal',
    'typevoorwerp',
    'typecd',
    'functievoorwerp',
    'versiering',
    'beschrijving',
    'opmerkingen',
    'doos'],
   'grid': 6}),
 ('Datering',
  {'columns': [{'fields': ['datering_vanaf',
      'datering_tot',
      'artefactdatering_vanaf',
      'artefactdatering_tot'],
     'grid': 6},
    {'fields': ['vondstdatering_vanaf',
      'vondstdatering_tot',
      'spoordatering_vanaf',
      'spoordatering_tot'],
     'grid': 6}]}),
 ('Migratie-informatie',
  {'fields': ['soort', 'brondata', 'uuid'], 'expanded': False})]

In [11]:
for i in range(0, len(show_fieldsets)-1):
    print(show_fieldsets[i])

('Projectvelden', {'columns': [{'fields': ['project', 'vondst', 'subnr', 'aantal', 'artefactsoort', 'abr_materiaal', 'sub_abr_materiaal', 'typevoorwerp', 'typecd', 'functievoorwerp', 'versiering', 'beschrijving', 'opmerkingen', 'doos'], 'grid': 6}, {'fields': ['fotos'], 'grid': 6, 'fulldisplay': True}]})
('Algemene Artefactvelden', {'columns': [{'fields': ['origine', 'conservering', 'exposabel', 'restauratie', 'literatuur', 'publicatiecode', 'catalogus', 'weggegooid'], 'grid': 6}, {'fields': ['aantal', 'gewicht', 'afmetingen', 'formaat_horizontaal', 'formaat_vericaal', 'compleetheid', 'mai', 'diversen'], 'grid': 6}]})
('Datering', {'columns': [{'fields': ['datering_vanaf', 'datering_tot', 'artefactdatering_vanaf', 'artefactdatering_tot'], 'grid': 6}, {'fields': ['vondstdatering_vanaf', 'vondstdatering_tot', 'spoordatering_vanaf', 'spoordatering_tot'], 'grid': 6}]})
